# Notes
---

## Equations:

### Gausian (Normal) Distribution
- $\mathcal{N}(x;\mu,\sigma^2)$
- $p(x) = (2\pi\sigma^{2})^{-\frac{1}{2}}\ exp\{-\frac{1}{2} \frac{(x-\mu)^2}{\sigma^2} \}$
- $p(x) = det(2\pi\Sigma)^{-\frac{1}{2}}\ exp\{-\frac{1}{2} (x-\mu)^T \Sigma^-1 (x-\mu) \}$ (Multivariate Distribution)

### Independence
- $p(x,y) = p(x)\ p(y)$
- $p(x\ |\ y) = \frac{p(x,y)}{p(y)} = \frac{p(x)\ p(y)}{p(y)} = p(x)$

### Conditional Independence
- When y does not cary information about x if z is known:
- $p(x\ |\ z) = p(x\ |\ z,y)$
- $p(y\ |\ z) = p(y|z,x)$
- This does not imply absolute indepence of x and y

### Total Probability
- $p(x) = \Sigma_y p(x\ |\ y)p(y)$ (discrete)
- $p(x) = \int p(x\ |\ y)p(y) dy$ (continuous)

### Bays Rule
- $p(y\ |\ x) = \frac{p(x\ |\ y)p(x)}{p(y)} = \frac{p(x\ |\ y)p(x)}{\Sigma_{x'}p(y\ |\ x')p(x')}$ (discrete)
- $p(y\ |\ x) = \frac{p(x\ |\ y)p(x)}{p(y)} = \frac{p(x\ |\ y)p(x)}{\int p(y\ |\ x')p(x') dx'}$ (continuous)
- $p(y\ |\ x) = \eta \space p(x\ |\ y)p(x)$ ($\eta$ is a normalizer)
- $p(x\ |\ y,z) = \frac{p(y\ |\ x,z)p(x\ |\ z)}{p(y\ |\ x)}$
- $p(x,y\ |\ z) = p(x\ |\ z)p(y\ |\ z)$

### Expectation
- $E[X] = \Sigma_x x \space p(x)$ (discrete)
- $E[X] = \int x \space p(x) \space dx$ (continuous)
- $E[aX+b] = a \space E[X] + b$

### Covariance
- The squared expected deviatino from the mean
- $Cov[X] = E[X - E[X]]^2 = E[X^2] - E[X]^2$

### Entropy
- The expected informationh that the value x carries
- $H_p (x) = E[-log_2(p(x))]$
- $H_p (x) = -\Sigma_x p(x) log_2(p(x))$
- $H_p (x) = -\int p(x) \space log_2(p(x)) \space dx$

---
## Acronyms
- PDF = Probability Density Function
- HMM = Hidden Markov Model
- DBN = Dynamic Bayes Network

### Definitions

---
## Robot Environment Interaction

- State = $x$ - Collection of all aspects of the robot and the environment that can affect the future
    - $x_t$ is the state $x$ at time $t$
- Pose - Robot location, orientation, and configuration
- Control Action = $u$
    - Control Actions decreases information about the state
- Measurement = $z$
    - Measurement increases information about the state
- Belief = $bel(x_t)$ - Belief about the state $x$ at time $t$
    - $\overline{bel}(x_t)$ - Belief about the state $x$ at time $t$ before incorperating measurement $z_t$
- State Update
    - Assume state $x_t$ provides sufficient information about all previous states, control actions, and measurements
    - Assume the robot makes a control action, then takes a measurement
    - State Transition Probability $p(x_t\ |\ x_{t-1}, u_t)$
    - Measurement Probability $p(z_t\ |\ x_{t})$
- Probability Distribuion
    - Prediction $\overline{bel}(x_t)$
        - Predicts the state after Control Action
    - Correction $bel(x_t)$
        - Corrects state estimation using measurement information

---

## Bayes Filter

1. **Algorithm Bayes_filter**($bel(x_t-1), u_t, z_t$):
1. $\ \ $for all $x_t$ do
1. $\ \ $$\ \ $$\overline{bel}(x_t) = \int\ p(x_t\ |\ u_t, x_{t-1})\ bel(x_{t-1})\ dx_{t-1}$
1. $\ \ $$\ \ $$bel(x_t) = \eta\ p(z_t\ |\ x_t)\ \overline{bel}(x_t)$
1. $\ \ $end for
1. $\ \ $return $bel(x_t)$


- Step 3 is prediction
- Step 4 is the measurement update (correction)
- The value of $bel(x_t)$ is fed back into the algorithm for the next iteration
- The final result is normaized to 1 by normalization constant $\eta$
- $bel(x_0)$ must be given
    - If is is known, it should be a point mass
    - If it is NOT known, it should be a uniform distribution
    - If partial knowledge, it should be a some non-uniform distribution (not common in practice)
- This simple form only works for very simple estimation problems

---
## Markov Assumption
- The past and future states are independent if you know the current state $x_t$
- Will not be true for almost any problem we work on, but the approximation is generally good enough

# Exercises

## 1. Robot Range Finder

- Range finder measures between 0m and 3m
- Assume the probability of measureing distances is evently distributed
- If the range finder is faultly, it will always measures < 1m
- Prior probability of sensor being faultly is p=0.01
- If the sensor is meausered N times and the value is always below 1m, what is the posterior probability that the sensor is faulty?
- Use Bayes's theorm:
    - $p(bad\_sensor\ |\ measurement) = \frac{p(measurement\ |\ bad\_sensor)\ p(bad\_sensor)}{p(measurement)}$

In [3]:
import numpy as np
import pandas as pd
pd.set_option('float_format', lambda x: '{:.4f}'.format(x))

# Initialize the constants
prior_prob_bad = 0.01
prob_meas_lt_1_given_bad = 1
prob_meas_lt_1_given_good = 1 / 3

# Initialize the probability that the sensor is bad
prob_bad = prior_prob_bad

probabilites = []

# Loop over the measurements
for N in range(10+1):
    
    # Save the probability
    probabilites.append(prob_bad)
    
    # Compute the probability of the measurement being less than 1
    prob_good = 1 - prob_bad  # Probablity the sensor is good
    prob_meas = prob_bad  * prob_meas_lt_1_given_bad + \
                prob_good * prob_meas_lt_1_given_good

    # Update the probability that the sensor is bad
    prob_bad = (prob_meas_lt_1_given_bad * prob_bad) / prob_meas


# Print the result
result = pd.DataFrame(data=np.array([probabilites]))
result.index = ['p(bad|N measurements)',]
result.columns.name = 'Samples'
result

Samples,0,1,2,3,4,5,6,7,8,9,10
p(bad|N measurements),0.0100,0.0294,0.0833,0.2143,0.4500,0.7105,0.8804,0.9567,0.9851,0.9950,0.9983


## 2. Weather simulation

- The weather can take on three values, sunny, cloudy, and rainy
- The weather transition function is a Markov chain with this transition table


In [4]:
transition_table = pd.DataFrame(np.array([[.8, .2, 0], [.4, .4, .2], [.2, .6, .2]]))
transition_table.columns=pd.MultiIndex.from_arrays([['tomorrow']*3, ['sunny', 'cloudy', 'rainy']])
transition_table.columns.names = ['when', 'weather']
transition_table.index = pd.MultiIndex.from_arrays([['today']*3, ['sunny', 'cloudy', 'rainy']])
transition_table.index.names = ['when', 'weather']
transition_table

when          tomorrow              
weather          sunny cloudy  rainy
when  weather                       
today sunny     0.8000 0.2000 0.0000
      cloudy    0.4000 0.4000 0.2000
      rainy     0.2000 0.6000 0.2000

### (a)
- Suppose day 1 = sunny
- Find the probability of the following sequnce of days:
    - Day 2 = cloudy
    - Day 3 = cloudy
    - Day 4 = rainy

In [5]:
todays_weather = 'sunny'
forcast = ['cloudy', 'cloudy', 'rainy']
prob_of_sequence = 1

for tomorrows_weather in forcast:
    prob_of_transition = transition_table['tomorrow', tomorrows_weather].loc['today', todays_weather]
    prob_of_sequence *= prob_of_transition
    todays_weather = tomorrows_weather

print('probablity of sequence = {:.2f}%'.format(prob_of_sequence*100))

probablity of sequence = 1.60%


### (b)
- Write a simulator that can randomly generate sequences of "weather" using this state transition matrix

In [7]:
import random

class WeatherSimulator:
    '''Class that simulates the weather given a transition table and starting weather.'''
    
    def __init__(self, transition_table, start_weather):
        self.todays_weather = start_weather
        
        # Precomute the cumulative transition probability
        cum_probs = dict()
        for weather in transition_table.loc['today'].index: # all possible weather states
            next_weather = transition_table.loc['today'].loc[weather]['tomorrow'] # all possible transitions
            next_weather_states = next_weather.index.values # names (eg. sunny)
            next_weather_cum_probs = np.cumsum(next_weather).values # cumulative sum of probabilities
            cum_probs[weather] = (next_weather_states, next_weather_cum_probs)
        self.cum_probs = cum_probs
        
    def simulate(self):
        '''Choses a next weather state at random given the transition probabilities.
            The simulator state is updated to this new state and this new state is returned.'''
        possible_weather, cumulative_sum = self.cum_probs[self.todays_weather]
        r = random.random()
        #print(r, cumulative_sum, cumulative_sum > r, possible_weather[cumulative_sum > r])
        tomorrows_weather = possible_weather[cumulative_sum > r][0]
        self.todays_weather = tomorrows_weather
        return tomorrows_weather
    
ws = WeatherSimulator(transition_table, 'sunny')
print([ws.simulate() for i in range(10)])

['cloudy', 'cloudy', 'sunny', 'cloudy', 'cloudy', 'sunny', 'sunny', 'sunny', 'sunny', 'sunny']


### (c)
- Use the simulator to come up with the steady state weather probabilities

In [8]:
# sunny, cloudy, rainy
weather_states = transition_table.loc['today'].index.values

# Initalize the results dict
results = dict()
for weather_state in weather_states:
    results[weather_state] = 0

# Initalize the weather simulator
ws = WeatherSimulator(transition_table, 'sunny')

# Run N iterations and count the times the weather was in each state
N = 100000
for i in range(N):
    results[ws.simulate()] +=1

# Print the results
for weather in weather_states:
    print('p({}) = {:.2f}%'.format(weather, results[weather]*100.0/N))

p(sunny) = 64.26%
p(cloudy) = 28.64%
p(rainy) = 7.10%


### (d)
- Find the closed form solution for the steady state weather probabilities
- We can write the system of equations:
    - $p(sunny) = \Sigma_{weather}\ p(sunny\ |\ weather)$
        - S = (.8S + .4C + .2R)
        - 0 = -.2S + .4C + .2 R
    - $p(cloudy) = \Sigma_{weather}\ p(cloudy\ |\ weather)$
        - C = (.2S + .4C + .6R)
        - 0 = .2S + -.6C + .6R
    - 1 = $\Sigma_{weather}\ p(weather)$
        - 1.0 = S + C + R
- Sovling this systems of equations for S,C,R will yeild the steady state probabilities
        

In [9]:
weather_states = ['sunny', 'cloudy', 'rainy']

a = np.array([[-0.2, 0.4, 0.2],
              [0.2, -0.6, 0.6],
              [1.0, 1.0, 1.0]])
b = np.array([0.0,0.0,1.0])

# Solve the system of equations
p_sunny, p_cloudy, p_rainy = np.linalg.solve(a,b)

# Print the results
for weather in weather_states:
    print('p({}) = {:.2f}%'.format(weather, results[weather]*100.0/N))

p(sunny) = 64.26%
p(cloudy) = 28.64%
p(rainy) = 7.10%


### (e)
- Solve for the entropy of this system
- entropy = -∑p(x)*log2(p(x))

In [10]:
# Grab the results from (d)
probabilities = np.array([p_sunny, p_cloudy, p_rainy])

entropy = -sum(probabilities * np.log2(probabilities))

print('entropy = {:.4f}'.format(entropy))

entropy = 1.1981


## 3. Noisy Sensor

In [ ]:
# [sunny, cloudy, rainy]
p_w_0 = np.array([1.0,0.0,0.0])

def weather_update(w):
    """Returns the probabiliy of tomorrows weather given the probablity to todays weather."""
    t = np.array([[0.8, 0.2, 0.0],
                  [0.4, 0.4, 0.2],
                  [0.2, 0.6, 0.2]])
    return np.dot(w,t)

def normalize(p):
    """Normailzes the probablities to they add up to One"""
    return p / p.sum()

def measure_update(w, m):
    """Returns the probabiliy of todays weather given the prior probablity to todays weather and a measurement."""
    t = np.array([[0.6, 0.4, 0.0],
                  [0.3, 0.7, 0.0],
                  [0.0, 0.0, 1.0]]).T
    
    raw = w*t[m]
    return raw / raw.sum()


In [ ]:
ms = [1,1,2,0]
w = p_w_0[:]
for i, m in enumerate(ms):
    print 'day %d, m = %d' % (i + 2, m)
    w = weather_update(w)
    print '\t prior = %s' % str(w)
    w = measure_update(w,m)
    print '\t post = %s' % str(w)


## 4. Continuous Estimation

In [ ]:
def gauss(mu, sigma_sq, x):
    return (2*np.pi*sigma_sq)**-0.5 *np.exp(-(x-mu)**2/(2*sigma_sq))

gauss(0,np.sqrt(0.1),0.1)